# Course Overview:
1. Algorithm Trading Basic
2. API Basics and Course Configuration

## Projects:

1. Equal-weight S&P 500 Screener
2. Quantitative Momentum Screener
3. Quantitative Value Screener

### Introduction
Algorithm Trading means using computers to make investment decisions.

Python is the most popular language used in quantitative trading. However, it is slow and sometimes is used to trigger functionality that actually runs in other faster programming languages.

The process of running a quantitative investing strategy can be broken down into the following steps:
1. Collect data
2. Develop a hypothesis for a strategy
3. Backtest that strategy
4. Implement the strategy in production

Because this is an introductory couse, it will differ from production algorithmic trading in 3 major ways:
1. We will be using random data
2. We will not be executing trades
3. We will be saving recommended trades into Excel files

### What is an API?
An API is an Application Programming Interface. APIs allow you to interact with someone else's software using your own code

In this couse, we will be using the IEX Cloud API to gather stock market data to make investment decisions.
API Functionality:
1. GET: get data from API
2. POST: add data to the database exposed by the API
3. PUT: add and overwrite data in the database exposed by the API
4. DELETE: delete data from the API's database


In [ ]:
symbol = 'AAPL'
api_url = f''
data = requests.get(api_url).json()

## Project 1: Equal-weight S&P 500 Screener

S&P 500 is the world's most popular stock market index. Many investment funds are benchmarked to the S&P 500 (top 500 stocks in the US). This means that they seek to replicate the performance of this index by owning all stocks that are held in the index.

One of the most important characteristics of the S&P 500 is that it is market capitalization-weighted. This means that larger companies get a correspondingly larger weight in the index.

In the first project, you will build an alternative version of the S&P 500 where each company has the same weighting.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constitute you should purchase to get an equal-width version of the index fund.

In [9]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [14]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks['Ticker']

0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
       ... 
500     YUM
501     ZBH
502    ZBRA
503    ZION
504     ZTS
Name: Ticker, Length: 505, dtype: object

Acquiring an API Token & GET requests from API

In [18]:
def get_price_and_market_cap(symbol):
    IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
    api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"

    data = requests.get(api_url).json()

    # get price and market of a particular stock
    price = data['latestPrice']
    market_cap = data['marketCap']
    return price, market_cap

In [15]:
# adding stocks data to pd 
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_df = pd.DataFrame(columns = my_columns)

for stock in stocks['Ticker']:
    price, market_cap = get_price_and_market_cap(stock)
    final_df = final_df.append(pd.Series([stock, price, market_cap, 'N/A'], index=my_columns), ignore_index=True)

KeyboardInterrupt: 

In [42]:
final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,153.62,45766682272,N/A
1,AAL,21.10,13827317945,N/A
2,AAP,213.86,14396615171,N/A
3,AAPL,151.30,2490497133113,N/A
4,ABBV,121.60,215293874673,N/A
...,...,...,...,...
500,YUM,122.00,36038373425,N/A
501,ZBH,162.09,34221265674,N/A
502,ZBRA,574.33,29413310254,N/A
503,ZION,53.25,8923541749,N/A


Using Batch API Calls to Improve Performance

In [16]:
# Dividing a list into chunks of n objects
def chunks(lst, n):
    '''Yield successive n-sized chunks from lst'''
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [19]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

stock_groups = list(chunks(stocks['Ticker'], 100))
stock_strings = []
for i in range(0, len(stock_groups)):
    stock_strings.append(','.join(stock_groups[i]))

final_df = pd.DataFrame(columns = my_columns)
for stock_string in stock_strings:
    batch_api_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={stock_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_url).json()
    for stock in stock_string.split(','):
        final_df = final_df.append(pd.Series([stock, data[stock]['quote']['latestPrice'], data[stock]['quote']['marketCap'], 'N/A'], index=my_columns), ignore_index=True)

In [20]:
final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,152.16,47389765842,N/A
1,AAL,20.96,13977567215,N/A
2,AAP,218.57,14638363042,N/A
3,AAPL,146.96,2463263662992,N/A
4,ABBV,119.08,206164490774,N/A
...,...,...,...,...
500,YUM,119.47,36535885714,N/A
501,ZBH,162.78,33459593870,N/A
502,ZBRA,560.15,30510040989,N/A
503,ZION,53.44,8922582388,N/A


In [37]:
portfolio_size = input('Enter value of portfolio: ')

try:
    portfolio_size = float(portfolio_size)
    print(portfolio_size)
except ValueError:
    print('Value of portfolio is not of type number\n')
    portfolio_size = input('Enter value of portfolio: ')
    portfolio_size = float(portfolio_size)
    print(portfolio_size)


Enter value of portfolio: 1000000
1000000.0


In [38]:
position_size = portfolio_size/len(final_df.index)

for i in range(len(final_df.index)):
    final_df.at[i, 'Number of Shares to Buy'] = position_size//final_df.at[i, 'Stock Price']

In [39]:
final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,152.16,47389765842,13.0
1,AAL,20.96,13977567215,94.0
2,AAP,218.57,14638363042,9.0
3,AAPL,146.96,2463263662992,13.0
4,ABBV,119.08,206164490774,16.0
...,...,...,...,...
500,YUM,119.47,36535885714,16.0
501,ZBH,162.78,33459593870,12.0
502,ZBRA,560.15,30510040989,3.0
503,ZION,53.44,8922582388,37.0


Format to Excel Output

In [77]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_df.to_excel(writer, 'Recommended Trades', index=False)

In [78]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

    
writer.save()

## Project 2: Momentum Investing Project 

Imagine that you have the choice between investing in two stocks that have had the following returns over the last year:
- AAPL: 35%
- MSFT: 20%

A momentum onvesting strategy would suggest investing in Apple because of its higher recent price return. There are many other nuances to momentum investing strategies that we will explore later.

'Momentum investing' means investing in the stocks that have increased in price the most.
For this project, we are going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-width portfolio of these 50 stocks.

### Feature Engineering - HQM Score
High-quality momentum stocks show slow and steady outperformance over long periods of time.
Low quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be caused by short-term news that is unlikely to be repeated in the future.

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest precentiles of:
- 1-month price returns 
- 3-month price returns
- 6-month price returns
- 1-year price returns

Metric used in this project to select top stocks is HQM (High-quality Momentum Score). HQM Score is the arithmetic mean of the 4 momentum percentile scores that we calculated. Then pick the top 50 best momentum stocks based on HQM Score.

In [4]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

In [16]:
stocks = pd.read_csv('sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

symbol = 'AAPL'
stat = 'year5ChangePercent'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2535846821125,
 'week52high': 149.88,
 'week52low': 90.17,
 'week52highSplitAdjustOnly': 150.59,
 'week52lowSplitAdjustOnly': 89.16,
 'week52change': 0,
 'sharesOutstanding': 17514119719,
 'float': 0,
 'avg10Volume': 101549360,
 'avg30Volume': 82941723,
 'day200MovingAvg': 132.22,
 'day50MovingAvg': 136.59,
 'employees': 151513,
 'ttmEPS': 4.61,
 'ttmDividendRate': 0.8626062774488287,
 'dividendYield': 0.005791726255699585,
 'nextDividendDate': '',
 'exDividendDate': '2021-05-01',
 'nextEarningsDate': '2021-07-25',
 'peRatio': 33.195157268611695,
 'beta': 1.5832757781691458,
 'maxChangePercent': 56.32504359116035,
 'year5ChangePercent': 5.56931666830415,
 'year2ChangePercent': 1.9895764638730262,
 'year1ChangePercent': 0.5365855408274044,
 'ytdChangePercent': 0.1027748212045396,
 'month6ChangePercent': 0.12117428011795553,
 'month3ChangePercent': 0.0859222055577578,
 'month1ChangePercent': 0.1442162609427004,
 'day30ChangePercent': 0.149915021

In [15]:
data['year1ChangePercent']

0.5385044993986591

In [18]:
# Dividing a list into chunks of n objects
def chunks(lst, n):
    '''Yield successive n-sized chunks from lst'''
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [73]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

stock_groups = list(chunks(stocks['Ticker'], 100))
stock_strings = []
for i in range(0, len(stock_groups)):
    stock_strings.append(','.join(stock_groups[i]))

my_columns = ['Ticker', 'Price', 'One-year Price Return', 'Number of Shares to Buy']

final_df = pd.DataFrame(columns = my_columns)

for stock_string in stock_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={stock_string},fb&types=quote,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for stock in stock_string.split(','):          
        final_df = final_df.append(pd.Series([stock, data[stock]['quote']['latestPrice'], data[stock]['stats']['year1ChangePercent'], 'N/A'], index=my_columns), ignore_index=True)

In [74]:
final_df

,Ticker,Price,One-year Price Return,Number of Shares to Buy
0,A,153.16,0.696441,N/A
1,AAL,21.65,0.721815,N/A
2,AAP,211.85,0.58079,N/A
3,AAPL,154.27,0.558079,N/A
4,ABBV,120.38,0.271408,N/A
...,...,...,...,...
500,YUM,118.81,0.376208,N/A
501,ZBH,162.32,0.31293,N/A
502,ZBRA,553.03,1.123384,N/A
503,ZION,52.99,0.616007,N/A


Remove Low-Momentum Stocks and drop out all stocks outside top 50

In [75]:
final_df.sort_values('One-year Price Return', ascending=False, inplace=True)
final_df = final_df.iloc[:50]
final_df.reset_index(inplace=True)
final_df

,index,Ticker,Price,One-year Price Return,Number of Shares to Buy
0,275,LB,77.08,3.569415,N/A
1,441,TPR,42.80,2.228386,N/A
2,148,DVN,28.30,2.132478,N/A
3,208,GPS,32.00,1.860884,N/A
4,179,FCX,34.91,1.825257,N/A
5,410,SIVB,597.67,1.733373,N/A
6,129,DFS,123.69,1.732222,N/A
7,106,COF,163.33,1.715549,N/A
8,272,KSS,54.28,1.669324,N/A
9,317,MRO,12.98,1.611721,N/A


In [76]:
portfolio_size = input('Enter portfolio size: ')

try:
    portfolio_size = float(portfolio_size)
except ValueError:
    print('Error! Enter numeric values.\n')
    portfolio_size = input('Enter portfolio size: ')
    portfolio_size = float(portfolio_size)

Enter portfolio size: 1000000


In [86]:
money_per_stock = portfolio_size/50

for i in range(len(final_df.index)):
    final_df.at[i, 'Number of Shares to Buy'] = money_per_stock//final_df.at[i, 'Price']

In [87]:
final_df

,index,Ticker,Price,One-year Price Return,Number of Shares to Buy
0,275,LB,77.08,3.569415,259.0
1,441,TPR,42.80,2.228386,467.0
2,148,DVN,28.30,2.132478,706.0
3,208,GPS,32.00,1.860884,625.0
4,179,FCX,34.91,1.825257,572.0
5,410,SIVB,597.67,1.733373,33.0
6,129,DFS,123.69,1.732222,161.0
7,106,COF,163.33,1.715549,122.0
8,272,KSS,54.28,1.669324,368.0
9,317,MRO,12.98,1.611721,1540.0


### Better Momentum Strategy:

High quality momentum when price returns change slowly and steadily

In [171]:
hqm_columns = ['Ticker', 
               'Price',
              'Number of Shares to Buy',
              'One-Year Price Return',
              'One-Year Return Percentile',
              'Six-Month Price Return',
              'Six-Month Return Percentile',
              'Three-Month Price Return',
              'Three-Month Return Percentile',
              'One-Month Price Return',
              'One-Month Return Percentile',
               'HQM Score'
              ]


IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

stock_groups = list(chunks(stocks['Ticker'], 100))
stock_strings = []
for i in range(0, len(stock_groups)):
    stock_strings.append(','.join(stock_groups[i]))

hqm_df = pd.DataFrame(columns = hqm_columns)

for stock_string in stock_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={stock_string},fb&types=quote,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for stock in stock_string.split(','):          
        hqm_df = hqm_df.append(pd.Series(
            [
              stock,
                data[stock]['quote']['latestPrice'], 
                'N/A',
                data[stock]['stats']['year1ChangePercent'], 
                'N/A',
                data[stock]['stats']['month6ChangePercent'], 
                'N/A',
                data[stock]['stats']['month3ChangePercent'], 
                'N/A',
                data[stock]['stats']['month1ChangePercent'], 
                'N/A',
                'N/A'
                
            ],
            index=hqm_columns), ignore_index=True)

In [172]:
hqm_df.fillna(0, inplace=True)

time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_df.index:
    for time_period in time_periods:
        hqm_df.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_df[f'{time_period} Price Return'], hqm_df.loc[row, f'{time_period} Price Return'])

HQM Score is the arithmetic mean of those 4 percentile score.

In [173]:
from statistics import mean

for row in hqm_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{time_period} Return Percentile'])
    hqm_df.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,150.75,N/A,0.672015,74.257426,0.184885,61.386139,0.128122,79.009901,0.030876,71.683168,71.584158
1,AAL,21.16,N/A,0.790472,83.564356,0.260875,79.405941,-0.079731,10.49505,-0.105710,4.356436,44.455446
2,AAP,211.40,N/A,0.531441,61.782178,0.254716,77.623762,0.131166,80.19802,0.066418,88.118812,76.930693
3,AAPL,155.31,N/A,0.559535,63.366337,0.161574,56.831683,0.134285,81.188119,0.147964,99.207921,75.148515
4,ABBV,116.86,N/A,0.252142,28.118812,0.077355,31.881188,0.124429,78.415842,0.028799,71.089109,52.376238
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,120.68,N/A,0.360453,39.405941,0.096436,38.415842,-0.004677,34.653465,-0.008241,49.90099,40.594059
501,ZBH,158.04,N/A,0.273175,29.90099,-0.041466,7.920792,-0.075405,11.485149,-0.011183,46.732673,24.009901
502,ZBRA,542.86,N/A,1.037196,91.287129,0.290057,83.960396,0.060349,60.792079,0.041615,77.623762,78.415842
503,ZION,53.90,N/A,0.660992,73.465347,0.021667,18.811881,-0.083577,9.306931,-0.050629,20.0,30.39604


In [174]:
hqm_df = hqm_df.sort_values('HQM Score', ascending=False).iloc[:50]
hqm_df.reset_index(drop=True,inplace=True)
hqm_df

In [177]:
portfolio_size = input('Enter portfolio size: ')

try:
    portfolio_size = float(portfolio_size)
except ValueError:
    print('Error! Please enter numeric value')
    portfolio_size = input('Enter portfolio size: ')
    portfolio_size = float(portfolio_size)

Enter portfolio size: 100000


In [178]:
money_per_stock = portfolio_size/50
for row in range(len(hqm_df.index)):
    hqm_df.loc[row, 'Number of Shares to Buy'] = money_per_stock//hqm_df.loc[row, 'Price']

In [179]:
hqm_df

,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,251,IT,258.912,7.0,1.148536,94.059406,0.578547,99.207921,0.351943,99.80198,0.112736,97.623762,97.673267
1,195,FTNT,263.290,7.0,0.940075,88.316832,0.766388,100.0,0.288875,98.415842,0.108278,97.029703,95.940594
2,345,NVDA,798.300,2.0,0.918222,87.722772,0.504767,98.217822,0.312460,99.207921,0.101296,95.841584,95.247525
3,275,LB,76.260,26.0,3.587643,100.0,0.675873,99.60396,0.133202,80.990099,0.149798,99.405941,95.0
4,477,WAT,379.272,5.0,0.902611,86.930693,0.432839,96.435644,0.256736,98.217822,0.109737,97.227723,94.70297
5,236,IDXX,662.150,3.0,0.964725,89.306931,0.355937,91.881188,0.296545,98.811881,0.116828,98.019802,94.50495
6,173,EXR,179.600,11.0,0.836428,85.544554,0.593431,99.405941,0.256495,98.019802,0.080893,91.881188,93.712871
7,436,TGT,254.830,7.0,1.176239,94.455446,0.297914,84.950495,0.245915,97.425743,0.090151,93.861386,92.673267
8,481,WELL,88.642,22.0,0.859439,86.336634,0.415748,96.039604,0.189146,91.683168,0.089175,93.465347,91.881188
9,243,INTU,527.180,3.0,0.787657,83.366337,0.396313,94.257426,0.235985,96.237624,0.061676,86.534653,90.09901


Then, an excel format can be exported if desired

## Project 3: Quantitative Value Screener

Value investing means investing in stocks that are trading below their perceived instrisic value.
Creating algorithmic value investing strategies relies on a concept called mulitples.

Multiples are calculated by dividing a company's stock price by some measure of the company's worth-like earnings or assets.

Here are a few examples of common multiples used in value investing:
- Price-to-earnings
- Price-to-book-value
- Price-to-free-cash-flow

Each of the individual multiples used by value investors has its pros and cons. One way to minimize the impact of any specific multiple is by using a composite.

We will use a composite of 5 different value metrics in our strategy.

In [180]:
import pandas as pd
import numpy as np
import xlsxwriter
import requests
from scipy import stats
import math

In [182]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [189]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

price = data['latestPrice']
 

In [202]:
# Dividing a list into chunks of n objects
def chunks(lst, n):
    '''Yield successive n-sized chunks from lst'''
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

stock_groups = list(chunks(stocks['Ticker'], 100))
stock_strings = []
for i in range(0, len(stock_groups)):
    stock_strings.append(','.join(stock_groups[i]))

my_columns = ['Ticker', 'Price', 'PE Ratio', 'Number of Shares to Buy']

final_df = pd.DataFrame(columns = my_columns)

for stock_string in stock_strings:
    batch_api_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={stock_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_url).json()
    
    for stock in stock_string.split(','):          
        final_df = final_df.append(pd.Series([stock, data[stock]['quote']['latestPrice'], data[stock]['quote']['peRatio'], 'N/A'], index=my_columns), ignore_index=True)

In [207]:
final_df.sort_values('PE Ratio',ascending=False,inplace=True)
final_df = final_df[final_df['PE Ratio'] > 0]
final_df = final_df[:50]
final_df.reset_index()

,Ticker,Price,PE Ratio,Number of Shares to Buy
109,COP,60.250,3349.15,N/A
42,APA,19.560,911.96,N/A
339,NOW,577.340,780.84,N/A
160,EOG,78.600,723.08,N/A
206,GPC,135.660,367.38,N/A
...,...,...,...,...
459,UNM,27.889,7.3,N/A
17,AFL,55.059,7.04,N/A
503,ZION,52.260,6.55,N/A
95,CINF,123.480,6.5,N/A


Remove 'glamour' stocks: sort the